<a href="https://colab.research.google.com/github/Awais-mohammad/Sentiment-Analysis/blob/main/Test_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# libraries
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import csv
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import wordnet
import re

In [ ]:
# nlts download
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
data=pd.read_csv('drive/MyDrive/features/test.csv',header='infer')

In [ ]:
dataChunk=data[:3]
print(dataChunk)

    ReviewID                                         ReviewText  Sentiments
0  141745902  When checking in, I was informed that the two ...           0
1  444756169  My room key didn't work originally however the...           1
2   33242386  We called the hotel from the airport and asked...           1


In [ ]:
# preprocessing data
# method to implement stop words removal
filteredSentences = []
def removestop(reviews):
    stop_words = set(stopwords.words('english'))
    r_txt = reviews
    for i in range(0, len(r_txt)):
        sentence = []
        joinsentence = []
        word_tokens = r_txt[i].split()
        # print(word_tokens)
        for w in word_tokens:
            if w.lower() not in stop_words:
                sentence.append(w)
                joinsentence = (" ").join(sentence)
        filteredSentences.append(joinsentence)
    return filteredSentences

In [ ]:
removestop(dataChunk.ReviewText)

['checking in, informed two rooms booked (a single double) facing side wall instead â\x80\x9cmagnificent viewâ\x80 promised online. even called arriving Italy make sure booking ok unprecedented, hotel fully booked choice.The rooms clean breakfast nice great service, terms positives, it. air conditioning didnâ\x80\x99t work, beds many broken springs them, bed frame would move creak sitting upon like break (I weight 8 stone!), shower over-flowed onto bathroom floor making virtually impossible next person use it, bed sheets thread-bare holes them.The disappointing aspect trip, however, behaviour hotel guests response staff. 2.30am guests floor us screaming shouting made racket people floor woken well us. called reception desk repeatedly ask sort took hour respond. guests tried speak loud people took notice.Many people spoke early morning flights catch annoyed night disrupted. noise tolerable, continue hour without stopped unacceptable.',
 'room key work originally however GSA, Matt called

In [ ]:
# Pos tagging
tokened_Sen = []
def POS_taggs(rTxt):
    for i in range(0, len(rTxt)):
        token = word_tokenize(rTxt[i])
        afterTag = nltk.pos_tag(token)
        tokened_Sen.append(afterTag)
    return tokened_Sen

In [ ]:
POS_taggs(filteredSentences)

In [ ]:
lemmitized=[]
lemmatizer = WordNetLemmatizer()

In [ ]:
def get_wordnet_pos(treebank_tag):
    print(treebank_tag)
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
for i in range(0,len(tokened_Sen)):
    sentence = []
    joinsentence = []
    for j in range(0,len(tokened_Sen[i])):
        word=lemmatizer.lemmatize(tokened_Sen[i][j][0].lower(),get_wordnet_pos(tokened_Sen[i][j][1]))
        sentence.append(word)
        joinsentence = (" ").join(sentence) 
    lemmitized.append(joinsentence)

In [ ]:
afterNumer=[]
# method to ressolve numeronyms
def ressolveNumeronyms(r_txt):
    numer = {'G8': 'group of eight', '24/7': 'twenty-four seven', '24/7': 'twenty-four seven', 'WWII': 'world war two',
             'G7': 'group of seven',
             '143': 'i love bed', '3D': 'three dimentional', 'e2a': 'excellent',
             'b4a': 'personal effects accounting documents', 'b3a': 'bachelors of accounting'
             }

    for i in range(0, len(r_txt)):
        new = r_txt[i].split()
        for i in range(0, len(new)):
            for k in numer:
                if k == new[i]:
                    new[i] = numer[k]
        a = (' ').join(new)
        print(a)
        afterNumer.append(a)
    return afterNumer

In [ ]:
ressolveNumeronyms(lemmitized)

check in , inform two room book ( a single double ) face side wall instead âmagnificent viewâ promise online . even call arrive italy make sure book ok unprecedented , hotel fully book choice.the room clean breakfast nice great service , term positive , it . air condition didnât work , bed many broken spring them , bed frame would move creak sit upon like break ( i weight 8 stone ! ) , shower over-flowed onto bathroom floor make virtually impossible next person use it , bed sheet thread-bare hole them.the disappointing aspect trip , however , behaviour hotel guest response staff . 2.30am guest floor u scream shout make racket people floor wake well u . call reception desk repeatedly ask sort take hour respond . guest try speak loud people take notice.many people speak early morning flight catch annoy night disrupt . noise tolerable , continue hour without stop unacceptable .
room key work originally however gsa , matt call engineer second attempt immediately come rescue . issue go

['check in , inform two room book ( a single double ) face side wall instead â\x80\x9cmagnificent viewâ\x80 promise online . even call arrive italy make sure book ok unprecedented , hotel fully book choice.the room clean breakfast nice great service , term positive , it . air condition didnâ\x80\x99t work , bed many broken spring them , bed frame would move creak sit upon like break ( i weight 8 stone ! ) , shower over-flowed onto bathroom floor make virtually impossible next person use it , bed sheet thread-bare hole them.the disappointing aspect trip , however , behaviour hotel guest response staff . 2.30am guest floor u scream shout make racket people floor wake well u . call reception desk repeatedly ask sort take hour respond . guest try speak loud people take notice.many people speak early morning flight catch annoy night disrupt . noise tolerable , continue hour without stop unacceptable .',
 'room key work originally however gsa , matt call engineer second attempt immediately c

In [ ]:
for i in range(0,len(afterNumer)):
    mystr=afterNumer[i]
    pattern=re.compile(r'\B# | \B#')
    new=pattern.finditer(mystr)
    for n in new:
        if len(mystr) > n.start()+1:
            newstr=mystr[:n.start()] + mystr[n.start()+1].upper() + mystr[n.start()+2:]
            print(newstr)
            afterNumer[i]=newstr

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)
cleaned=[]
for i in afterNumer:
    cleaned.append(remove_tags(i))

In [ ]:
################################################### after preprocessing moving towards features ##############################################################

In [ ]:
cleaned

['check in , inform two room book ( a single double ) face side wall instead â\x80\x9cmagnificent viewâ\x80 promise online . even call arrive italy make sure book ok unprecedented , hotel fully book choice.the room clean breakfast nice great service , term positive , it . air condition didnâ\x80\x99t work , bed many broken spring them , bed frame would move creak sit upon like break ( i weight 8 stone ! ) , shower over-flowed onto bathroom floor make virtually impossible next person use it , bed sheet thread-bare hole them.the disappointing aspect trip , however , behaviour hotel guest response staff . 2.30am guest floor u scream shout make racket people floor wake well u . call reception desk repeatedly ask sort take hour respond . guest try speak loud people take notice.many people speak early morning flight catch annoy night disrupt . noise tolerable , continue hour without stop unacceptable .',
 'room key work originally however gsa , matt call engineer second attempt immediately c

In [ ]:
# libraries
import pandas as pd
import string
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from sklearn.feature_extraction.text import CountVectorizer
from heapq import nlargest
from gensim.models import Word2Vec

In [ ]:
featureMat=pd.DataFrame()

In [ ]:
semiColons=[]
questionMarks=[]
exclamations=[]
for i in range(0,len(cleaned)):
    newcount=[]
    semiCount=0
    exclamCount=0
    questionMark=0
    for j in range(0,len(cleaned[i])):
        if cleaned[i][j] == ':':
            semiCount=semiCount+1
        elif cleaned[i][j] == '!':
            exclamCount=exclamCount+1
        elif cleaned[i][j]=='?':
            questionMark=questionMark+1
    semiColons.append(semiCount)
    questionMarks.append(questionMark)
    exclamations.append(exclamCount)

In [ ]:
featureMat.insert(len(featureMat.columns), 'ExclamationsCount', exclamations)
featureMat.insert(len(featureMat.columns), 'QuestionMarkCount', questionMarks)
featureMat.insert(len(featureMat.columns), 'SemiColonsCount', semiColons)
featureMat.head()

In [ ]:
!pip install textblob 

In [ ]:
from textblob import TextBlob

In [ ]:
subjectivity=[]
for i in range(0,len(cleaned)):
    blob=TextBlob(cleaned[i])
    sub=blob.sentiment.subjectivity
    subjectivity.append(sub)
print(len(subjectivity))

3


In [ ]:
featureMat.insert(len(featureMat.columns), 'Subjectivity_Score', subjectivity)

In [ ]:
# Pos tagging
tokened_Sen=[]
def POS_taggs():
    for i in range(0, len(cleaned)):
        token = word_tokenize(cleaned[i])
        afterTag = nltk.pos_tag(token)
        tokened_Sen.append(afterTag)
    return tokened_Sen
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
POS_taggs()

[[('check', 'NN'),
  ('in', 'IN'),
  (',', ','),
  ('inform', 'VB'),
  ('two', 'CD'),
  ('room', 'NN'),
  ('book', 'NN'),
  ('(', '('),
  ('a', 'DT'),
  ('single', 'JJ'),
  ('double', 'NN'),
  (')', ')'),
  ('face', 'NN'),
  ('side', 'VBP'),
  ('wall', 'NN'),
  ('instead', 'RB'),
  ('â\x80\x9cmagnificent', 'JJ'),
  ('viewâ\x80', 'NNS'),
  ('promise', 'VBP'),
  ('online', 'NN'),
  ('.', '.'),
  ('even', 'RB'),
  ('call', 'VB'),
  ('arrive', 'JJ'),
  ('italy', 'NNS'),
  ('make', 'VBP'),
  ('sure', 'JJ'),
  ('book', 'NN'),
  ('ok', 'NN'),
  ('unprecedented', 'JJ'),
  (',', ','),
  ('hotel', 'NN'),
  ('fully', 'RB'),
  ('book', 'NN'),
  ('choice.the', 'NN'),
  ('room', 'NN'),
  ('clean', 'JJ'),
  ('breakfast', 'NN'),
  ('nice', 'JJ'),
  ('great', 'JJ'),
  ('service', 'NN'),
  (',', ','),
  ('term', 'NN'),
  ('positive', 'JJ'),
  (',', ','),
  ('it', 'PRP'),
  ('.', '.'),
  ('air', 'NN'),
  ('condition', 'NN'),
  ('didnâ\x80\x99t', 'NN'),
  ('work', 'NN'),
  (',', ','),
  ('bed', 'VBD'),
  

In [ ]:
positiveWords=[]
negativeWords=[]
neutralWords=[]
for i in range(0,len(tokened_Sen)):
    posCount=0
    negCount=0
    neuCount=0
    for j in range(0,len(tokened_Sen[i])):
        synsets=wn.synsets(tokened_Sen[i][j][0],pos=get_wordnet_pos(tokened_Sen[i][j][1]))
        if synsets:
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            if swn_synset.pos_score() > swn_synset.neg_score() and swn_synset.pos_score() > swn_synset.obj_score():
                posCount+=1
            elif swn_synset.neg_score() > swn_synset.pos_score() and swn_synset.neg_score() > swn_synset.obj_score():
                negCount+=1
            elif swn_synset.obj_score() > swn_synset.pos_score() and swn_synset.obj_score() > swn_synset.neg_score():
                neuCount+=1
    positiveWords.append(posCount)
    negativeWords.append(negCount)
    neutralWords.append(neuCount)

In [ ]:
featureMat.insert(len(featureMat.columns), 'positiveWords', positiveWords)
featureMat.insert(len(featureMat.columns), 'negativeWords', negativeWords)
featureMat.insert(len(featureMat.columns), 'neutralWords', neutralWords)
featureMat.head()

ValueError: ignored

In [ ]:
finale=[]
for i in range(0,len(cleaned)):
    tokens=word_tokenize(cleaned[i])
    sentences=[]
    joined=[]
    for word in tokens:
        if word.isalpha() and len(word) > 2:
            sentences.append(word)
        joined=" ".join(sentences)
    finale.append(joined)
len(finale)

3

In [ ]:
a=pd.Series(finale)
a=a.map(''.join)

In [ ]:
CountVec = CountVectorizer(ngram_range=(1,1),stop_words='english')
Count_data = CountVec.fit_transform(a)
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_dataframe.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,air,airport,annoy,area,arrive,ask,aspect,attempt,bacon,...,unacceptable,unprecedented,use,virtually,wait,wake,walk,wall,weight,work
0,0,1,0,1,0,1,1,1,0,0,...,1,1,1,1,0,1,0,1,1,1
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,1,0,2,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
add={}
for i in cv_dataframe.columns:
    add[i]=cv_dataframe[i].sum()

In [ ]:
top10 = nlargest(10, add, key = add.get)
top10

['room',
 'hotel',
 'bar',
 'ask',
 'bed',
 'book',
 'breakfast',
 'floor',
 'great',
 'guest']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)    
X = vectorizer.fit_transform(a)
len(vectorizer.get_feature_names())
X=X.astype(np.uint8)
X=X.todense()
tdf=pd.DataFrame(X,columns=vectorizer.get_feature_names())
tdf.head(20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,air,airport,annoy,area,arrive,ask,aspect,attempt,back,...,wake,walk,wall,weight,well,within,without,work,would,you
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# for the top 10 thin
valDict={}
for i in top10:
    valDict[i]=tdf[i]
tfIDF=pd.DataFrame(valDict)
tfIDF.head()

,room,hotel,bar,ask,bed,book,breakfast,floor,great,guest
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0


In [ ]:
featureMat.index = tfIDF.index
updatedMatrix=pd.concat([featureMat, tfIDF], axis=1)

In [ ]:
!pip install gensim

In [ ]:
ser=pd.Series(cleaned)
tokenedData=ser.apply(lambda x:x.split())
model = Word2Vec(sentences=ser,size=100, window=5,workers=4,sg=1)
model.train(ser,total_examples=len(ser),epochs=14)

(4551, 25620)

In [ ]:
def makeFeatureVec(words):
#     print(words)
    feature_vec = np.zeros((100,),dtype="float32") 
    for word in words:
        try:
            feature_vec = np.add(feature_vec,model.wv[word])
        except:
            continue
    return feature_vec
def avgVectors(reviews):
    counter = 0
    review_feature_vecs = np.zeros((len(reviews),100), dtype='float32')
    for review in reviews:
        review_feature_vecs[counter] = makeFeatureVec(review)
        counter = counter + 1
    return review_feature_vecs

In [ ]:
trainDataVecs = avgVectors(cleaned)

In [ ]:
word2VecFrame = pd.DataFrame(trainDataVecs)

In [ ]:
namesDict={}
for i in range(0,len(word2VecFrame.columns)):
  namesDict[word2VecFrame.columns[i]]= 'word2Vec' + str(word2VecFrame.columns[i]) 
word2VecFrame.rename(columns = namesDict, inplace = True)

In [ ]:
updatedMatrix.index = word2VecFrame.index
featuresMat=pd.concat([featureMat, word2VecFrame], axis=1)

In [ ]:
featuresMat

,ExclamationsCount,QuestionMarkCount,SemiColonsCount,Subjectivity_Score,positiveWords,negativeWords,neutralWords,word2Vec0,word2Vec1,word2Vec2,...,word2Vec90,word2Vec91,word2Vec92,word2Vec93,word2Vec94,word2Vec95,word2Vec96,word2Vec97,word2Vec98,word2Vec99
0,1,0,0,0.574914,3,4,93,86.809799,75.774940,-52.281208,...,11.505980,-20.491989,2.683457,-39.911461,-182.332260,-106.988853,-25.360683,-78.786285,-85.401642,98.893898
1,1,0,0,0.646875,0,3,27,24.611383,21.494400,-14.851165,...,3.236446,-5.778812,0.773100,-11.305467,-51.851768,-30.387161,-7.171172,-22.408722,-24.283678,28.037033
2,1,0,0,0.439474,4,1,76,66.499519,57.949409,-40.116425,...,8.966373,-15.679215,2.173076,-30.451765,-139.628525,-81.767532,-19.309048,-60.332039,-65.504517,75.714760


In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 25.8 MB/s 
     |████████████████████████████████| 462 kB 55.3 MB/s 


In [ ]:
# libraries
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
import pandas as pd

In [ ]:
# bert preprocessorhttps://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert encoder https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2
# encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4",trainable=True)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2",trainable=True)

In [ ]:
# preprocessing dataset adding cls sep etc 
# slicing reviews due to unsufficient resources
inputs = preprocessor(cleaned)
# feeding it to model for vectorization
outputs = encoder(inputs)

In [ ]:
temp=pd.DataFrame()
for i in range(0,len(outputs['sequence_output'])):
  b=outputs['sequence_output'][i].numpy().sum(axis=0)
  temp=temp.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [ ]:
temp.index = featuresMat.index
featureMat=pd.concat([temp, featuresMat], axis=1)
featureMat.head()

,0,1,2,3,4,5,6,7,8,9,...,word2Vec90,word2Vec91,word2Vec92,word2Vec93,word2Vec94,word2Vec95,word2Vec96,word2Vec97,word2Vec98,word2Vec99
0,41.510395,35.285660,-5.240277,-33.980553,15.383636,-8.030009,95.334404,9.543857,-45.058701,-14.178946,...,11.505980,-20.491989,2.683457,-39.911461,-182.332260,-106.988853,-25.360683,-78.786285,-85.401642,98.893898
1,55.255733,3.440856,-17.169939,-35.548088,-29.446119,-34.797497,1.062917,-19.378189,4.180927,-1.396281,...,3.236446,-5.778812,0.773100,-11.305467,-51.851768,-30.387161,-7.171172,-22.408722,-24.283678,28.037033
2,66.850044,2.580216,29.234505,-20.586494,-8.552543,-37.084141,94.446205,2.881490,-32.878109,-1.455382,...,8.966373,-15.679215,2.173076,-30.451765,-139.628525,-81.767532,-19.309048,-60.332039,-65.504517,75.714760


In [ ]:
featureMat.insert(len(featureMat),'Sentiments',dataChunk.Sentiments)

In [ ]:
featureMat.to_csv('drive/MyDrive/features/testFeatures')

In [ ]:
import pickle
data=featureMat.drop('Sentiments',axis=1)

In [ ]:
import pandas as pd
fM=pd.read_csv('drive/MyDrive/features/testFeatures')